In [1]:
import sys
sys.path.append("../../../")
sys.path.append('../../')
import os
import time
import json
import torch
from tqdm import tqdm
from itertools import cycle
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
import argparse
from torch.nn import functional as F
import matplotlib.pyplot as plt


from potential_flows import transforms, potential, encoders, data, flow
from potential_flows.data import get_dataset, create_custom_dataset
from flow.arguments import set_seed, parse_arguments, parse_notebook_arguments, get_parser

In this notebook, we study the dimension and sample size complexity of dual and minimax algorithms. 

1. For **dimension dependence**, we run the Trainer for both methods for 10 steps and report the average CPU time.

2. For **training sample size dependence**, we report the average time for evaluating the objective function for both methods.

### Dimension Complexity

In [2]:
torch.manual_seed(42)
args = parse_notebook_arguments()

dim_list = [int(10*(n+1)) for n in range(100)]
time_list = torch.zeros(len(dim_list))
args.num_steps = 10

In [3]:
## Dual Method ##

# running 10 training steps for each dimension

for i,d in enumerate(dim_list):
    
    # create datasets and dataloaders for dimension d

    args.data_shape = (int(d),)
    dataset_x, dataset_y = get_dataset(args, split="train")
    test_x, test_y = get_dataset(args, split="test")

    data_loader_X = DataLoader(dataset_x, batch_size=args.batch_size, shuffle=True)
    data_loader_Y = DataLoader(dataset_y, batch_size=args.batch_size, shuffle=True)

    # create flow

    tail_bound = torch.max(torch.cat([torch.abs(dataset_x.data), torch.abs(dataset_y.data)]))
    potential_flow = potential.ICRQ(tail_bound=args.tail_factor*tail_bound, num_bins=args.num_bins, data_shape=args.data_shape)

    # train for 10 steps

    OT_Trainer = flow.DualOT_Trainer(potential_flow, args, dataset_x=data_loader_X, dataset_y=data_loader_Y, test_x=test_x, test_y=test_y)
    
    start = time.perf_counter(), time.process_time()
    OT_Trainer.learn()
    stop = time.perf_counter(), time.process_time()
    
    time_list[i] = stop[1] - start[1]

torch.save({'dim': dim_list, 'time': time_list/args.num_steps}, 'dual_dimension_complexity.pt')

# plotting dimension vs per training step time in seconds

plt.figure(figsize=(5,4))
plt.plot(dim_list, time_list/args.num_steps)
plt.xlabel('Dimension')
plt.ylabel('Time per train step')

    

  0%|          | 0/10 [00:00<?, ?it/s]/home/medhaaga/miniconda3/envs/nsf/lib/python3.10/site-packages/torch/autograd/__init__.py:394: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  result = Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
 10%|█         | 1/10 [00:28<04:19, 28.83s/it]

In [ ]:
## Minmax Method ##

# running 10 training steps for each dimension

for i,d in enumerate(dim_list):
    
    # create dataset and dataloaders for dimension d
    
    args.data_shape = (int(d),)
    dataset_x, dataset_y = get_dataset(args, split="train")
    test_x, test_y = get_dataset(args, split="test")

    data_loader_X = DataLoader(dataset_x, batch_size=args.batch_size, shuffle=True)
    data_loader_Y = DataLoader(dataset_y, batch_size=args.batch_size, shuffle=True)
    
    # create potential flows for f and g

    tail_bound = torch.max(torch.cat([torch.abs(dataset_x.data), torch.abs(dataset_y.data)]))
    potential_flow_x = potential.ICRQ(tail_bound=args.tail_factor*tail_bound, num_bins=args.num_bins, data_shape=args.data_shape)
    potential_flow_y = potential.ICRQ(tail_bound=args.tail_factor*tail_bound, num_bins=args.num_bins, data_shape=args.data_shape)

    ## train

    OT_Trainer = flow.MinmaxOT_Trainer(potential_flow_x, potential_flow_y, args, dataset_x=data_loader_X, dataset_y=data_loader_Y, test_x=test_x, test_y=test_y)       
    
    start = time.perf_counter(), time.process_time()
    OT_Trainer.learn()
    stop = time.perf_counter(), time.process_time()

    time_list[i] = stop[1] - start[1]
    print(f'Dimension: {d}, Time: {time_list[i]}')
    
torch.save({'dim': dim_list, 'time': time_list/args.num_steps}, 'minmax_dim_complexity.pt')

plt.plot(dim_list, time_list/args.num_steps)
plt.xlabel('Dimension')
plt.ylabel('Time per train step')


### Sample Complexity

In [ ]:
# Sample complexity

args.data_shape = (2,)
n_list = [int(1000*(n+1)) for n in range(100)]
time_list = np.zeros(len(n_list))

# create potential flows for f and g

potential_flow_x = potential.ICRQ(tail_bound=2, num_bins=args.num_bins, data_shape=args.data_shape)
potential_flow_y = potential.ICRQ(tail_bound=2, num_bins=args.num_bins, data_shape=args.data_shape)


In [ ]:
# running 10 training steps for each dimension

for i,n in enumerate(tqdm(n_list)):
    
    args.num_samples = int(n)
    dataset_x, dataset_y = get_dataset(args, split="train")
    test_x, test_y = get_dataset(args, split="test")

    data_loader_X = DataLoader(dataset_x, batch_size=args.batch_size, shuffle=True)
    data_loader_Y = DataLoader(dataset_y, batch_size=args.batch_size, shuffle=True)

    OT_Trainer = flow.DualOT_Trainer(potential_flow, args, dataset_x=data_loader_X, dataset_y=data_loader_Y, test_x=test_x, test_y=test_y)
    
    start = time.perf_counter(), time.process_time()
    OT_Trainer.objective(dataset_x.data, dataset_y.data)
    stop = time.perf_counter(), time.process_time()
    
    time_list[i] = stop[1]-start[1]

torch.save({'dim': n_list, 'time': time_list/args.num_steps}, 'dual_N_complexity.pt')

# plotting dimension vs per training step time in seconds

plt.figure(figsize=(5,4))
plt.plot(n_list, time_list/args.num_steps)
plt.xlabel('"Sample size (n)')
plt.ylabel('Average time for objective evaluation')

In [ ]:
## Minmax Method ##

# evaluating the objective 10 times for each sample size

for i,d in enumerate(n_list):
    
    # create dataset and dataloaders for dimension d
    
    args.num_samples = int(n)
    dataset_x, dataset_y = get_dataset(args, split="train")
    test_x, test_y = get_dataset(args, split="test")

    data_loader_X = DataLoader(dataset_x, batch_size=args.batch_size, shuffle=True)
    data_loader_Y = DataLoader(dataset_y, batch_size=args.batch_size, shuffle=True)
    
    ## train

    OT_Trainer = flow.MinmaxOT_Trainer(potential_flow_x, potential_flow_y, args, dataset_x=data_loader_X, dataset_y=data_loader_Y, test_x=test_x, test_y=test_y)       
    
    start = time.perf_counter(), time.process_time()
    OT_Trainer.objective(dataset_x.data, dataset_y.data)
    stop = time.perf_counter(), time.process_time()
    
    time_list[i] = stop[1] - start[1]
    print(f'Dimension: {d}, Time: {time_list[i]}')
    
torch.save({'dim': dim_list, 'time': time_list/args.num_steps}, 'minmax_dim_complexity.pt')

plt.plot(n_list, time_list/args.num_steps)
plt.xlabel('Dimension')
plt.ylabel('Time per train step')
